In [3]:
from qiskit_nature.units import *   #ANGSTROM
from qiskit_nature.second_q.drivers import *    #PySCFDriver
from qiskit_nature.second_q.mappers import * #JordanWignerMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.problems import EigenstateResult
from qiskit_nature.second_q.circuit.library import *  #Ansatz, HF
#from qiskit_ibm_runtime import QiskitRuntimeService, Session, Estimator, Options
from qiskit_algorithms.optimizers import * 
from qiskit_algorithms import VQE
from braket.tracking import Tracker
from qiskit_braket_provider import *
from qiskit_aer import StatevectorSimulator
from qiskit.primitives import Estimator
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_algorithms import AdaptVQE

import numpy as np
import matplotlib.pyplot as plt

In [25]:
def co2_molecula(z,y):
    driver = PySCFDriver(atom=f"O 0 {y} -{z}; C 0 0 0; O 0 {y} {z}", basis="sto-6g")
    estrutura_inicial = driver.run() 

    espaco_target_minimal = ActiveSpaceTransformer(4,4) 
    estrutura_minimal = espaco_target_minimal.transform(estrutura_inicial)
    mapa_minimal = estrutura_minimal.get_tapered_mapper(BravyiKitaevMapper())

    ansatz = UCCSD(
    estrutura_minimal.num_spatial_orbitals,
    estrutura_minimal.num_particles,
    mapa_minimal,
    initial_state=HartreeFock(
        estrutura_minimal.num_spatial_orbitals,
        estrutura_minimal.num_particles,
        mapa_minimal))


    vqe_solver = VQE(Estimator(), ansatz, SLSQP())
    vqe_solver.initial_point = [0.0] * ansatz.num_parameters


    calculo = GroundStateEigensolver(mapa_minimal, vqe_solver)
    resultado = calculo.solve(estrutura_minimal)
    return resultado

In [16]:
r = np.arange(1.10,1.21,0.01)
theta = np.arange(np.pi/4, np.pi/2 + np.pi/40, np.pi/40 )

In [26]:
par = [[r,o] for r in r for o in theta]

#info = [z,y,resultado]
info = [[a[0]*np.sin(a[1]),a[0]*np.cos(a[1]), co2_molecula(a[0]*np.sin(a[1]), a[0]*np.cos(a[1]))] for a in par]        

In [43]:
energia = []

for i in range(len(info)):
    energia.append([par[i][0],par[i][1],info[i][2].total_energies[0]])

In [60]:
raio = [energia[i][0] for i in range(len(energia))]
ang = [energia[i][1] for i in range(len(energia))]
energ = [energia[i][2] for i in range(len(energia))]
z = [info[i][0] for i in range(len(info))]
y = [info[i][1] for i in range(len(info))]

In [61]:
import pandas as pd

data = {'raio': raio,
        'angulo': ang,
        'coord_z': z,
        'coord_y': y,
        'energia': energ}

df = pd.DataFrame(data)
df.to_csv('PES do CO2.csv', index=False)